## Datasets Overview
For this capstone project, I will be utilizing several datasets from Yelp and Yummly in identifying correct restaurants through user's search input.

### Yelp Dataset
The [Yelp dataset](https://www.yelp.com/dataset) contains wealth of information on users and businesses spanning 10 metropolitan cities (Montreal, Calgary, Toronto, Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, and Cleveland).
- 8,021,122 reviews
- 209,393 businesses
- 1,320,761 tips

Prior to data wrangling, I utilized json_to_csv.py to flatten nested json objects within business' attributes column (link: [json_to_csv_converter](https://github.com/Yelp/dataset-examples/blob/master/json_to_csv_converter.py))

### Yummly Dataset
The [Yummly dataset](https://www.yummly.com/insights/understanding-cuisines) consists of 20 cuisines with various ingredients list per data row. 
- 20 cuisine types
- 39774 ingredients list

In [2]:
import pandas as pd
import numpy as np

# Webscraping for menu data
import requests
from bs4 import BeautifulSoup

# Ignore warning messages
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Utilize helper functions
%run helper_function.ipynb

## Import Files

In [4]:
# Yelp dataset
business_df = pd.read_csv('./dataset/business.csv')
review_df = pd.read_csv('./dataset/review.csv')
tip_df = pd.read_csv('./dataset/tip.csv')

# Yummly dataset
menu_df = pd.read_json('./dataset/cuisine_desc.json')

## 1) Preprocess Yummly Dataset
- Consolidate all ingredient list per cuisine type
- Apply tf-idf vectorizer to consolidated dataset

In [4]:
menu_df.shape

(39774, 3)

In [5]:
menu_df.isnull().any()

id             False
cuisine        False
ingredients    False
dtype: bool

Yummly dataset consists of 20 cuisine types along with its ingredient list.

In [6]:
# Yummly dataset
menu_df.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Top 5 most accounted cuisines in Yummly dataset are Italian, Mexican, American, Indian, and Chinese

In [7]:
# Yummly dataset
menu_df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

### 1.1 Consolidate all ingredients list per cuisine type
- Consolidating all text values per cuisine in preparation for tf-idf vectorizer

In [8]:
# Convert row ingredients as string
menu_df['item_list'] = menu_df['ingredients'].apply(lambda row: ','.join(map(str, row)))

# Consolidate all ingredients using groupby method
menu_df = menu_df.groupby('cuisine', as_index=False).agg({'item_list': ' '.join})

In [9]:
menu_df

,cuisine,item_list
0,brazilian,"ice cubes,club soda,white rum,lime,turbinado e..."
1,british,"greek yogurt,lemon curd,confectioners sugar,ra..."
2,cajun_creole,"herbs,lemon juice,fresh tomatoes,paprika,mango..."
3,chinese,"low sodium soy sauce,fresh ginger,dry mustard,..."
4,filipino,"eggs,pepper,salt,mayonaise,cooking oil,green c..."
5,french,"sugar,salt,fennel bulb,water,lemon olive oil,g..."
6,greek,"romaine lettuce,black olives,grape tomatoes,ga..."
7,indian,"water,vegetable oil,wheat,salt black pepper,sh..."
8,irish,"cooking spray,salt,black pepper,yukon gold pot..."
9,italian,"sugar,pistachio nuts,white almond bark,flour,v..."


### 1.2 Apply TF-IDF vectorizer to consolidated Yummly dataset
TF-IDF vectorizer is used to identify words or ingredients that have more at certain cuisine type while not appearing on other cuisines. It'll be useful when each cuisines' have its own relevant keywords so that during machine learning phase (word2vec, doc2vec, etc.); we can utilize this dataset to accurately respond to user's search request.
- Create vocabulary for tfidf
- Invoke helper function to create two menu tfidf's dataframes
    - Create one with using menu list provided by Yummly's dataset
    - Create second tfidf matrix dataframe using n-gram

#### 1.2.1 Create vocabulary for tfidf

In [10]:
# Create menu lists by converting concatenated string to list
menu_lists = []
menu_df['item_list'].apply(lambda text: menu_lists.append(text.split(',')))

# Flatten nested lists
menu_list = [item.strip() for menu_list in menu_lists for item in menu_list]

# Get unique values
menu_list = list(set(menu_list))

# Sample
menu_list[:10]

['',
 'coconut milk fresh basil',
 'orange olive oil',
 'ajwain',
 'vanilla extract fresh spinach',
 'salt lamb shanks',
 'beef broth flour',
 'italian seasoning large egg whites',
 'coriander orange',
 'Sangiovese']

In [11]:
# Remove first index which is an empty string
menu_list.pop(0)

''

#### 1.2.2 Invoke helper function to create menu tfidf's dataframes

#### Menu TF-IDF dataframe based on Yummly's given vocabulary

In [12]:
# Invoking function to return tfidf matrix dataframe
menu_tfidf_df = create_tfidf_matrix_df(menu_df, menu_list)

In [13]:
# Viewing result
menu_tfidf_df

,coconut milk fresh basil,orange olive oil,ajwain,vanilla extract fresh spinach,salt lamb shanks,beef broth flour,italian seasoning large egg whites,coriander orange,Sangiovese,chocolate-hazelnut spread,...,salt chopped celery,lemon juice cream of celery soup,white sesame seeds water,corn starch top sirloin steak,coffee liqueur seasoned bread crumbs,taro leaf,ground cumin chicken legs,coconut milk sweet rice,cajun seasoning garlic powder,frozen popcorn chicken
cuisine,,,,,,,,,,,,,,,,,,,,,
brazilian,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
british,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cajun_creole,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chinese,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
filipino,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
french,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
greek,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
indian,0.0,0.0,0.005134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
irish,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Checking the tf-idf result provided by Yummly's vocabulary

In [14]:
menu_tfidf_df.loc['indian'].sort_values(ascending=False)[:10]

salt         0.378676
oil          0.321795
garlic       0.259369
cumin        0.247644
masala       0.246614
ginger       0.233702
pepper       0.223878
onions       0.205024
coriander    0.183000
cilantro     0.165413
Name: indian, dtype: float64

In [15]:
menu_tfidf_df.loc['korean'].sort_values(ascending=False)[:10]

sesame    0.432001
oil       0.374249
sauce     0.308584
garlic    0.293430
onions    0.258071
soy       0.249806
pepper    0.246591
sugar     0.224090
rice      0.190109
salt      0.174497
Name: korean, dtype: float64

In [16]:
menu_tfidf_df.loc['mexican'].sort_values(ascending=False)[:10].keys().tolist()

['pepper',
 'salt',
 'garlic',
 'cheese',
 'oil',
 'cilantro',
 'chicken',
 'lime',
 'onions',
 'tortillas']

#### Identifying top 100 ingredients in all 20 cuisines

In [17]:
data = {}
for cuisine in menu_tfidf_df.index:
    # Top 100 ingredients that have high tf-idf score
    values = menu_tfidf_df.loc[cuisine].sort_values(ascending=False)[:100].keys().tolist()
    data[cuisine] = values

menu_vocab_tfidf_df = pd.DataFrame(data,columns=menu_tfidf_df.index.tolist())

In [18]:
menu_vocab_tfidf_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,pepper,salt,pepper,sauce,sauce,salt,pepper,salt,salt,cheese,pepper,sauce,sesame,pepper,pepper,salt,salt,pepper,sauce,sauce
1,oil,flour,garlic,oil,garlic,pepper,oil,oil,flour,pepper,salt,oil,oil,salt,oil,sugar,pepper,oil,lime,sugar
2,salt,sugar,salt,soy,pepper,butter,salt,garlic,butter,oil,garlic,mirin,sauce,garlic,salt,flour,sugar,salt,oil,garlic
3,cachaca,butter,onions,garlic,salt,sugar,lemon,cumin,sugar,salt,thyme,soy,garlic,cheese,cumin,butter,butter,garlic,garlic,oil
4,milk,eggs,oil,pepper,oil,oil,garlic,masala,pepper,garlic,oil,sugar,onions,oil,garlic,pepper,flour,red,fish,fish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,soda,crust,nutmeg,loin,breast,asparagus,pastry,chiles,rye,wheat,sage,tuna,paprika,honey,oregano,radishes,margarine,apples,linguine,papaya
96,kale,leeks,cilantro,almonds,loin,sage,salad,spices,pudding,seeds,ice,steak,perilla,beer,cauliflower,ketchup,sausages,starch,mussels,avocado
97,almonds,chives,crabmeat,spinach,paprika,veal,flatbread,peanuts,chips,porcini,cabbage,crab,gingerroot,wheat,stew,ginger,pimentos,hazelnuts,chestnuts,coffee
98,scallions,canola,rolls,cayenne,macaroni,shrimp,marjoram,beef,syrup,ravioli,browning,beans,roast,lard,wheat,shallots,bananas,salsa,cardamom,pineapple


In [19]:
menu_vocab_tfidf_df.to_csv('./dataset/menu_vocab_tfidf.csv')

Based on the results given by Menu TF-IDF's dataframe based on Yummly's vocabulary set - the result seemed general. For instance 'garlic', 'salt', and 'pepper' are visible on all three cuisines. It did not provide any unique ingredient per cuisine as I had hoped. Therefore, I'll be creating a second Menu tf-idf dataframe based on n-gram.

#### Menu TF-IDF dataframe based on N-gram

In [20]:
# Invoking function to return tfidf matrix dataframe
menu_tfidf_df = create_tfidf_matrix_df(menu_df)

In [21]:
# Viewing result
menu_tfidf_df

,10,10 oz,10 oz frozen,14,14 oz,14 oz diced,14 oz sweetened,25,25 less,25 less sodium,...,zucchini vegetable,zucchini vegetable broth,zucchini vegetable oil,zucchini vegetable stock,zucchini whole,zucchini yellow,zucchini yellow bell,zucchini yellow onion,zucchini yellow squash,épices
cuisine,,,,,,,,,,,,,,,,,,,,,
brazilian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
british,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cajun_creole,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001773,0.001773,0.001773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
chinese,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
filipino,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004968,0.000000,0.005695,0.000000,0.000000
french,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.003798,0.002503,0.002316,0.000000,0.000000,0.002020,0.003028,0.000000,0.000000,0.006056
greek,0.002727,0.002727,0.002727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
indian,0.000000,0.000000,0.000000,0.000864,0.000864,0.000000,0.000958,0.000000,0.000000,0.000000,...,0.001201,0.000792,0.000000,0.000958,0.000000,0.001916,0.000000,0.001465,0.000000,0.000000
irish,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004734,0.000000,0.005773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


TF-IDF model with n_gram range from 1 to 3 - produced 365373 columns which is a lot more than previous vocab based TF-IDF dataframe of 33727 columns

#### Checking the TF-IDF result based on ngrams

In [22]:
menu_tfidf_df.loc['indian'].sort_values(ascending=False)[:10]

garam masala         0.462462
garam                0.462462
curry leaves         0.201379
dal                  0.157047
ghee                 0.146125
coriander powder     0.120609
garam masala salt    0.108011
fenugreek            0.107102
cardamom pods        0.105446
masala salt          0.102902
Name: indian, dtype: float64

In [23]:
menu_tfidf_df.loc['korean'].sort_values(ascending=False)[:10]

gochujang               0.339206
gochujang base          0.339206
kimchi                  0.279347
toasted sesame seeds    0.206008
sauce sesame            0.169717
soy sauce sesame        0.160074
toasted sesame oil      0.148502
sauce sesame oil        0.129216
mirin                   0.128364
korean                  0.117417
Name: korean, dtype: float64

In [24]:
menu_tfidf_df.loc['mexican'].sort_values(ascending=False)[:10]

taco               0.288057
taco seasoning     0.259648
enchilada sauce    0.232524
enchilada          0.232524
jack cheese        0.214434
mexican            0.207057
refried beans      0.165673
refried            0.153691
tortilla chips     0.153636
tortilla           0.145080
Name: mexican, dtype: float64

#### Identifying top 100 ingredients in all 20 cuisines

In [25]:
data = {}
for cuisine in menu_tfidf_df.index:
    # Top 100 ingredients that have high tf-idf score
    values = menu_tfidf_df.loc[cuisine].sort_values(ascending=False)[:100].keys().tolist()
    data[cuisine] = values

menu_tfidf_df = pd.DataFrame(data,columns=menu_tfidf_df.index.tolist())

In [26]:
menu_tfidf_df

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
0,dried black beans,stilton,cajun,chinese,calamansi,gruyere cheese,feta cheese crumbles,garam masala,irish,lasagna,jerk,mirin,gochujang,taco,couscous,sauerkraut,grits,chorizo,thai,beansprouts
1,dried black,stilton cheese,cajun seasoning,oyster sauce,soy sauce bay,gruyere,cheese crumbles,garam,irish whiskey,lasagna noodles,jamaican,sake,gochujang base,taco seasoning,preserved lemon,buckwheat flour,whiskey,saffron threads,curry paste,thai
2,chocolate sprinkles,suet,andouille,shaoxing,oyster sauce,fresh tarragon,kalamata,curry leaves,irish cream liqueur,parmigiano,scotch bonnet,miso,kimchi,enchilada sauce,hanout,cottage cheese,bourbon,chorizo sausage,red curry,rice noodles
3,granola,currants,creole,hoisin,thai chile,swiss cheese,kalamata olives,dal,irish cream,parmigiano reggiano,bonnet chile,dashi,toasted sesame seeds,enchilada,ras,cottage,bourbon whiskey,spanish chorizo,red curry paste,vietnamese
4,ice lime,golden syrup,andouille sausage,hoisin sauce,thai,grated gruyère,pita,ghee,whiskey,reggiano cheese,scotch bonnet chile,nori,sauce sesame,jack cheese,ras el,buckwheat,collard greens,manchego cheese,lime leaves,thai basil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eggs parmesan,soda vanilla,onion powder dried,sesame oil rice,banana leaves,frisee,oregano black pepper,seeds cumin,fashioned,salt fresh basil,baking powder cornmeal,chili oil,sugar rice,ground cumin avocado,green olives olive,butter golden,chicken soup,salt manchego,sauce thai basil,sauce fresh ginger
96,passion fruit juice,sugar dark,powder dried oregano,sauce soy sauce,garlic coconut milk,chocolate large,yogurt garlic,salt ghee,wheat flour buttermilk,porcini,scallions coconut milk,soy sauce fresh,seeds green,flour tortillas salt,cinnamon lemon,potato starch,meal large,red peppers extra,paste lime,sauce fish
97,milk tapioca flour,potatoes all purpose,creole seasoning celery,sauce pork,sauce rice noodles,duck fat,pepper greek,jeera,carrots beef brisket,italian bread,scallions coconut,rice rice vinegar,reduced sodium soy,old el paso,lamb stock,potatoes all,cracker crumbs,tomatoes dry white,light soy,cilantro rice
98,ice granulated,sugar whipping cream,cajun seasoning all,rice vinegar corn,soy sauce vinegar,pernod,kalamata feta,gram flour,bacon pork sausages,cheese italian seasoning,thyme hot,green tea powder,soy sauce vegetable,el paso,ground coriander chopped,tomatoes potatoes,marshmallows,hot smoked paprika,light soy sauce,sesame oil rice


In [27]:
menu_tfidf_df.to_csv('./dataset/menu_tfidf.csv')

Menu TF-IDF's dataframe based on ngrams approach provided better result in identifying list of less general ingredients per cuisine types.

### Summary of Yummly dataset
- Initial Yummly's dataset contains 20 cuisines with 39,774 rows with food ingredients.
- Consolidated Yummly's dataset based on cuisine type which reduced row count to 20 (cuisine count total)
- Applied TF-IDF Transformer onto consolidated Yummly's dataset in which n_grams (unigrams to trigrams) provided better distinction between cuisines compared to given Yummly's menu titles.

---

## 2) Extracting menu data through webscraping
I decided to use BeautfiulSoup to webscrape allmenus.com as it contains restaurant name, cuisine type, menu item, and its menu description. All the data needed to help supplement our current datasets in recommending restaurants to the users based on their search input.

1. Remove businesses from Canada and lowercase all city and state names.
2. Collect restaurants menu from allmenus.com using business dataframe's states.
3. Based on given restaurants name from allmenus, filter both allmenus and yelp's business dataset.

### 2.1 Remove businesses from Canada and lowercase all city names
Yelp dataset contains businesses mostly from USA and some from Canada, I am filtering out cities from Canada as I'm focusing on restaurants from USA.

In [6]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [7]:
# Get businesses from US states
business_df = business_df[business_df['state'].isin(states)]

In [8]:
business_df['state'].unique()

array(['NC', 'AZ', 'NV', 'IL', 'PA', 'WI', 'SC', 'OH', 'CA', 'TX', 'NY',
       'CO', 'GA', 'AL', 'UT', 'VT', 'WA', 'NE', 'MI', 'FL', 'AR', 'HI',
       'OR', 'AK', 'VA', 'CT', 'MO'], dtype=object)

In [9]:
# Lowercasing city names and states to be case-insensitive
business_df["city"] = business_df["city"].str.lower()
business_df['state'] = business_df['state'].str.lower()

Gathered businesses from 27 US states and its cities from Yelp's dataset.

### 2.2 Collect restaurants menu from allmenus.com using business dataframe's states
Collect top 5 cities from filtered states data and begin webscraping for restaurants' menu.

In [32]:
# Check business counts in five major cities in Nevada
data = []
states = business_df['state'].unique()

for state in states:
    
    # Get cities per state
    cities = business_df[business_df['state'] == state].groupby(by=['city'])
    
    # Get top 5 cities per state
    cities = cities['city'].count().sort_values(ascending=False)[:5].keys().tolist()
    
    # Create object per each state
    obj = {'state': state, 'cities': cities}
    
    # Append each state's cities data
    data.append(obj)

In [33]:
# Viewing sample data
data[:3]

[{'state': 'nc',
  'cities': ['charlotte', 'concord', 'matthews', 'huntersville', 'gastonia']},
 {'state': 'az',
  'cities': ['phoenix', 'scottsdale', 'mesa', 'tempe', 'chandler']},
 {'state': 'nv',
  'cities': ['las vegas',
   'henderson',
   'north las vegas',
   'boulder city',
   'n las vegas']}]

#### Extract restaurant data from allmenus.com
- Get restaurants' sub url links and its restaurant titles from allmenus.com
- Build menu dataframe using restaurants' links and titles

#### Get restaurants' sub url links and its titles

In [34]:
import pickle # Saving lists or data from allmenus.com

In [166]:
rest_links, rest_titles = collect_menu_links(data) # collects restaurant links from allmenus.com

In [169]:
# Saving files for later use
with open('./dataset/rest_links.pkl', 'wb') as f:
    pickle.dump(rest_links, f)
    
with open('./dataset/rest_titles.pkl', 'wb') as f:
    pickle.dump(rest_titles, f)

#### Remove states that did not appear in allmenus.com

In [230]:
# Divide restaurant links by state
links = [] # Append list per state
unavailable_states = [] # collect unavailable states

for state in states:
    temp_links = [link for link in rest_links if link.split('/')[1] == state]
    
    # Check if there are states not available in allmenus.com
    if len(temp_links) == 0:
        unavailable_states.append(state)
    else:
        links.append(temp_links)    

In [717]:
unavailable_states

['al', 'vt', 'ar', 'hi', 'ak']

In [718]:
# Remove five states that are not used in allmenus.com
updated_states = [state for state in states if state not in unavailable_states]

#### Organize restaurant links and titles by state using dictionary

In [233]:
links = [] # dividing restaurant links per state
data = {}

for state in updated_states:
    rest_list = [link for link in rest_links if link.split('/')[1] == state] # get restaurant links per state
    titles = rest_titles[:len(rest_list)] # get restaurant titles per state
    
    data[state] = [rest_list, titles] # key value pair - state<key> restaurant links and titles <value>
    rest_titles = rest_titles[len(rest_list):] # update list by removing recently added restaurant titles

#### Build menu dataframe using restaurant links and titles
For some reason, could not make requests all the restaurant links/titles data at once. Therefore, dividing requests by state.

In [247]:
nc_menu_df = build_menu_df(data['nc'][0], data['nc'][1]) # Build menus dataframe

In [243]:
menu_df = build_menu_df(data['nv'][0], data['nv'][1]) # Build menus dataframe

In [ ]:
nv_menu_df = menu_df

In [251]:
az_menu_df = build_menu_df(data['az'][0], data['az'][1]) # Build menus dataframe

In [254]:
il_menu_df = build_menu_df(data['il'][0], data['il'][1]) # Build menus dataframe

In [255]:
pa_menu_df = build_menu_df(data['pa'][0], data['pa'][1]) # Build menus dataframe

In [256]:
wi_menu_df = build_menu_df(data['wi'][0], data['wi'][1]) # Build menus dataframe

In [257]:
sc_menu_df = build_menu_df(data['sc'][0], data['sc'][1]) # Build menus dataframe

In [258]:
oh_menu_df = build_menu_df(data['oh'][0], data['oh'][1]) # Build menus dataframe

In [259]:
ca_menu_df = build_menu_df(data['ca'][0], data['ca'][1]) # Build menus dataframe

In [264]:
ny_menu_df = build_menu_df(data['ny'][0], data['ny'][1]) # Build menus dataframe

In [265]:
co_menu_df = build_menu_df(data['co'][0], data['co'][1]) # Build menus dataframe

In [266]:
ga_menu_df = build_menu_df(data['ga'][0], data['ga'][1]) # Build menus dataframe

In [267]:
ut_menu_df = build_menu_df(data['ut'][0], data['ut'][1]) # Build menus dataframe

In [268]:
wa_menu_df = build_menu_df(data['wa'][0], data['wa'][1]) # Build menus dataframe

In [269]:
ne_menu_df = build_menu_df(data['ne'][0], data['ne'][1]) # Build menus dataframe

In [270]:
mi_menu_df = build_menu_df(data['mi'][0], data['mi'][1]) # Build menus dataframe

In [271]:
fl_menu_df = build_menu_df(data['fl'][0], data['fl'][1]) # Build menus dataframe

In [272]:
or_menu_df = build_menu_df(data['or'][0], data['or'][1]) # Build menus dataframe

In [273]:
va_menu_df = build_menu_df(data['va'][0], data['va'][1]) # Build menus dataframe

In [274]:
ct_menu_df = build_menu_df(data['ct'][0], data['ct'][1]) # Build menus dataframe

In [275]:
mo_menu_df = build_menu_df(data['mo'][0], data['mo'][1]) # Build menus dataframe

After successfully webscraping all restaurants from the allmenus website, concat all menu dataframes into one and save consolidated menu dataframe.

In [300]:
# Concat
frames = [nc_menu_df, nv_menu_df, az_menu_df, il_menu_df, pa_menu_df, wi_menu_df, sc_menu_df, oh_menu_df, 
          ca_menu_df, ny_menu_df, co_menu_df, ga_menu_df, ut_menu_df, wa_menu_df, ne_menu_df, mi_menu_df, 
          fl_menu_df, or_menu_df, va_menu_df, ct_menu_df, mo_menu_df]

allmenus_df = pd.concat(frames)
allmenus_df.head()

,id,name,menu_titles,menu_desc,categories
0,32810,El Cancun,Hamburger with fries,,Mexican
1,32810,El Cancun,Child's Plate #1,One taco&comma rice & beans,Mexican
2,32810,El Cancun,Child's Plate #2,Mini burrito&comma rice & beans,Mexican
3,32810,El Cancun,Chicken Plate,Grilled strips of chicken & fries,Mexican
4,32810,El Cancun,French Fries,,Mexican


In [164]:
allmenus_df.shape

(1354448, 5)

Webscraping all restaurants' list from 22 states have generated 5 columns with 1,354,448 datapoints. 

In [11]:
# Lowercase restaurants' names from both yelp and allmenus dataset
allmenus_df['name'] = allmenus_df['name'].str.lower()
business_df['name'] = business_df['name'].str.lower()

In [12]:
# Get unique restaurant names (remove duplicates)
restaurants_list = list(set(allmenus_df['name'].tolist()))

# Have 8875 unique restaurants
len(restaurants_list)

8875

In [13]:
business_df.shape

(153843, 60)

#### Save accumulated allmenus dataset

In [39]:
allmenus_df.to_csv('./dataset/menu.csv')

In [14]:
allmenus_df = pd.read_csv('./dataset/menu.csv', index_col=0)
allmenus_df

,id,name,menu_titles,menu_desc,categories
0,32810,el cancun,Hamburger with fries,NaN,Mexican
1,32810,el cancun,Child's Plate #1,One taco&comma rice & beans,Mexican
2,32810,el cancun,Child's Plate #2,Mini burrito&comma rice & beans,Mexican
3,32810,el cancun,Chicken Plate,Grilled strips of chicken & fries,Mexican
4,32810,el cancun,French Fries,NaN,Mexican
...,...,...,...,...,...
7192,368311,tokyo grill,Hibachi Steak or Teriyaki Steak,NaN,Japanese
7193,368311,tokyo grill,Hibachi Scallop,NaN,Japanese
7194,368311,tokyo grill,Filet Mignon,NaN,Japanese
7195,368311,tokyo grill,Atlantic Salmon,NaN,Japanese


## 2.3 Filter both Yelp's business dataset and Allmenus' menu dataset
 - Only populate restaurants that exist in both Yelp's business dataset and Allmenus' menu dataset.
 - Combine all text data into one row per restaurants for both Yelp's business and Allmenus' menu dataset.

### 2.3.1 Only populate restaurants that exist in both Yelp's business dataset and Allmenus' menu dataset

In [15]:
# Filter businesses dataframe based on given allmenus' restaurants list
business_df = business_df[business_df['name'].isin(restaurants_list)]

# Filter restaurants' name from allmenus dataset compared to Yelp's business dataset
filtered_allmenus_df = allmenus_df[allmenus_df['name'].isin(business_df['name'])]

In [16]:
business_df.shape

(3404, 60)

In [17]:
filtered_allmenus_df.shape

(313954, 5)

In [18]:
len(set(filtered_allmenus_df.name.tolist()))

1199

In [19]:
len(set(business_df['name'].tolist()))

1199

Restaurants that appeared on both allmenus and Yelp's business dataset resulted in the following:
    - Removed 150,439 restaurant data from Yelp's business dataset (97% reduction).
    - Removed 1,040,494 restaurants from allmenus data (76.8% reduction).
    - Resulted in having 1199 unique restaurants.

### 2.3.2 Cleaning menu and business datasets
- Create menu dataframe:
    - Menu dataframe with duplicate restaurants but with unique menu titles and menu descriptions per row.
- Creating two busiess dataframes:
    - 1) Business dataframe with combined reviews per restaurant.
    - 2) Business dataframe with duplicate restaurants but with unique review per row.
- Handle NaNs and missing values if any.
- For consolidated (combined text values per restaurant), identify duplicated rows and combine further.
    - Remove duplicate categories within each row (restaurant).
- Create new columns based on categories values - set default value as 0.

#### Handling Missing Values
Fill in missing menu description with menu titles as menu titles tells about the food description as well.

In [20]:
filtered_allmenus_df

,id,name,menu_titles,menu_desc,categories
521,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,"Mexican, Seafood"
522,37789,cabo fish taco,Guacamole,"A house blend of avocado, lime juice, cilantro...","Mexican, Seafood"
523,37789,cabo fish taco,Crispy Calamari,"Beer battered calamari rings, flash-fried and ...","Mexican, Seafood"
524,37789,cabo fish taco,Baja Shrimp and Crab Dip,Creamy dip made with lump crabmeat and seasone...,"Mexican, Seafood"
525,37789,cabo fish taco,Salsa Trio,"Roasted Red Tomato Salsa, fresh Pineapple Mang...","Mexican, Seafood"
...,...,...,...,...,...
6546,52138,fuji,Chicken,NaN,Japanese
6547,52138,fuji,Filet Mignon,NaN,Japanese
6548,52138,fuji,Shrimp & Scallops,NaN,Japanese
6549,52138,fuji,Steak & Chicken,NaN,Japanese


In [21]:
df = filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]
df

,id,name,menu_titles,menu_desc,categories
564,37789,cabo fish taco,Chicken Fajita,NaN,"Mexican, Seafood"
565,37789,cabo fish taco,Shrimp Fajita,NaN,"Mexican, Seafood"
566,37789,cabo fish taco,Steak Fajita,NaN,"Mexican, Seafood"
567,37789,cabo fish taco,Veggie Fajita,NaN,"Mexican, Seafood"
568,37789,cabo fish taco,Salmon Baja Bowl,NaN,"Mexican, Seafood"
...,...,...,...,...,...
6546,52138,fuji,Chicken,NaN,Japanese
6547,52138,fuji,Filet Mignon,NaN,Japanese
6548,52138,fuji,Shrimp & Scallops,NaN,Japanese
6549,52138,fuji,Steak & Chicken,NaN,Japanese


Add menu_titles values to menu_desc as menu_titles values has enough menu description

In [22]:
filtered_allmenus_df.loc[filtered_allmenus_df['menu_desc'].isna(), 'menu_desc'] = df['menu_titles']
filtered_allmenus_df

,id,name,menu_titles,menu_desc,categories
521,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,"Mexican, Seafood"
522,37789,cabo fish taco,Guacamole,"A house blend of avocado, lime juice, cilantro...","Mexican, Seafood"
523,37789,cabo fish taco,Crispy Calamari,"Beer battered calamari rings, flash-fried and ...","Mexican, Seafood"
524,37789,cabo fish taco,Baja Shrimp and Crab Dip,Creamy dip made with lump crabmeat and seasone...,"Mexican, Seafood"
525,37789,cabo fish taco,Salsa Trio,"Roasted Red Tomato Salsa, fresh Pineapple Mang...","Mexican, Seafood"
...,...,...,...,...,...
6546,52138,fuji,Chicken,Chicken,Japanese
6547,52138,fuji,Filet Mignon,Filet Mignon,Japanese
6548,52138,fuji,Shrimp & Scallops,Shrimp & Scallops,Japanese
6549,52138,fuji,Steak & Chicken,Steak & Chicken,Japanese


In [23]:
filtered_allmenus_df.isna().sum()

id             0
name           0
menu_titles    1
menu_desc      1
categories     0
dtype: int64

In [24]:
filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]

,id,name,menu_titles,menu_desc,categories
659753,314775,station,NaN,NaN,"American (New), French, Vegetarian, Vegan"


Without any information regarding its menu title and menu description, the row is considered no use because it is difficult to decipher what the menu item is without those essential information, therefore, removing the single row.

In [25]:
filtered_allmenus_df = filtered_allmenus_df.dropna()
filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]

,id,name,menu_titles,menu_desc,categories


### Text preprocessing
- Expanding contractions
- Tokenizing
- Lemmatizing
- Lowercasing
- Removing stop words

In [26]:
# Expanding contractions - ex: 'y'all can't' => 'you all cannot'
filtered_allmenus_df['clean_menu_desc'] = filtered_allmenus_df['menu_desc'].apply(lambda text: expand_contractions(text))

In [27]:
filtered_allmenus_df['clean_menu_titles'] = filtered_allmenus_df['menu_titles'].apply(lambda text: expand_contractions(text))

In [28]:
filtered_allmenus_df['clean_menu_desc'] = filtered_allmenus_df['menu_desc'].apply(lambda text: tokenize(text))

In [29]:
filtered_allmenus_df['clean_menu_titles'] = filtered_allmenus_df['menu_titles'].apply(lambda text: tokenize(text))

In [31]:
# Determine if ANY value in a series is missing
filtered_allmenus_df.isnull().values.any()

False

In [30]:
filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]

,id,name,menu_titles,menu_desc,categories,clean_menu_desc,clean_menu_titles


In [32]:
filtered_allmenus_df

,id,name,menu_titles,menu_desc,categories,clean_menu_desc,clean_menu_titles
521,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,"Mexican, Seafood",pipe hot melted cheese blend chipotle pepper s...,queso dip
522,37789,cabo fish taco,Guacamole,"A house blend of avocado, lime juice, cilantro...","Mexican, Seafood",a house blend avocado lime juice cilantro sign...,guacamole
523,37789,cabo fish taco,Crispy Calamari,"Beer battered calamari rings, flash-fried and ...","Mexican, Seafood",beer batter calamari ring flash fry serve roas...,crispy calamari
524,37789,cabo fish taco,Baja Shrimp and Crab Dip,Creamy dip made with lump crabmeat and seasone...,"Mexican, Seafood",creamy dip lump crabmeat seasoned shrimp serve...,baja shrimp crab dip
525,37789,cabo fish taco,Salsa Trio,"Roasted Red Tomato Salsa, fresh Pineapple Mang...","Mexican, Seafood",roasted red tomato salsa fresh pineapple mango...,salsa trio
...,...,...,...,...,...,...,...
6546,52138,fuji,Chicken,Chicken,Japanese,chicken,chicken
6547,52138,fuji,Filet Mignon,Filet Mignon,Japanese,filet mignon,filet mignon
6548,52138,fuji,Shrimp & Scallops,Shrimp & Scallops,Japanese,shrimp scallops,shrimp scallops
6549,52138,fuji,Steak & Chicken,Steak & Chicken,Japanese,steak chicken,steak chicken


There are no missing values.

#### Remove duplicate categories (ex: 'Chinese Chinese, Japanese')
Categories column has duplicate category; example shown below:

In [33]:
# Add Allmenus' categories onto business dataset matching by restaurant name
filtered_allmenus_df[filtered_allmenus_df['name'] == 'china gourmet']

,id,name,menu_titles,menu_desc,categories,clean_menu_desc,clean_menu_titles
11678,229355,china gourmet,20 oz. Coke,20 oz. Coke,Chinese,oz coke,oz coke
11679,229355,china gourmet,Pepsi,Pepsi,Chinese,pepsi,pepsi
11680,229355,china gourmet,20 oz. Dr Pepper,20 oz. Dr Pepper,Chinese,oz dr pepper,oz dr pepper
11681,229355,china gourmet,20 oz. Diet Dr Pepper,20 oz. Diet Dr Pepper,Chinese,oz diet dr pepper,oz diet dr pepper
11682,229355,china gourmet,20 oz. Diet Pepsi,20 oz. Diet Pepsi,Chinese,oz diet pepsi,oz diet pepsi
...,...,...,...,...,...,...,...
31230,294163,china gourmet,Kung Pao Prawn,Kung Pao Prawn,Chinese,kung pao prawn,kung pao prawn
31231,294163,china gourmet,Vegetable Prawn,Vegetable Prawn,Chinese,vegetable prawn,vegetable prawn
31232,294163,china gourmet,Taiwanese Bento,Taiwanese Bento,Chinese,taiwanese bento,taiwanese bento
31233,294163,china gourmet,Beef Noodle Soup,Beef Noodle Soup,Chinese,beef noodle soup,beef noodle soup


In [34]:
def get_unique_categories(categories):
    '''
    categories: str
        category list per restaurant
    Returns
    -------
    str : str
        Returns categories without duplicates
    '''
    value = categories.replace(',', ' ').split(' ') # Remove comma and split by space
    set_val = set(value) # Remove duplicates
    str_val = " ".join(map(str,set_val)) # Convert Set to string
    return str_val.strip() # Remove white space

In [35]:
# Apply that function to every row of the column
filtered_allmenus_df['categories'] = filtered_allmenus_df['categories'].apply(lambda x: get_unique_categories(x))

In [36]:
# Lowercase categories
filtered_allmenus_df['categories'] = filtered_allmenus_df['categories'].str.lower()
filtered_allmenus_df['categories']

521     seafood mexican
522     seafood mexican
523     seafood mexican
524     seafood mexican
525     seafood mexican
             ...       
6546           japanese
6547           japanese
6548           japanese
6549           japanese
6550           japanese
Name: categories, Length: 313953, dtype: object

Removed 142 duplicates by grouping futher with restaurant name.

#### Create new columns based on categories values - set default value as 0

In [37]:
from collections import Counter

In [38]:
def get_categories(df):
    # Creating new columns based on categorical values (ex: american, italian, japanese, etc.)
    categories = []

    for cat in df['categories']:
        for category in cat.split(' '):

            category = category.strip()

            if category != '' and category != '&':
                categories.append(category)

    return categories

In [39]:
categories = get_categories(filtered_allmenus_df)
filtered_menu_features = Counter(categories).most_common()
filtered_menu_features[:10] # Show top 10 categories

[('chinese', 121775),
 ('mexican', 83225),
 ('italian', 45155),
 ('japanese', 43284),
 ('asian', 38889),
 ('american', 33702),
 ('pizza', 27587),
 ('sushi', 27186),
 ('thai', 21325),
 ('pasta', 18590)]

In [ ]:
# Saving files for later use
with open('./dataset/features.pkl', 'wb') as f:
    pickle.dump(filtered_menu_features, f)

In [41]:
# Get unique categories
categories = list(set(categories))

# Concating dataframe with business_df
filtered_allmenus_df = pd.concat(
    [
        filtered_allmenus_df,
        pd.DataFrame(
            index = filtered_allmenus_df.index,
            columns = categories
        )
    ], axis = 1
)

filtered_allmenus_df.head()

,id,name,menu_titles,menu_desc,categories,clean_menu_desc,clean_menu_titles,chinese,bar,latin,...,dining,irish,nuevo,calzones,wraps,wings,greek,tex-mex,pakistani,american
521,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,seafood mexican,pipe hot melted cheese blend chipotle pepper s...,queso dip,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,37789,cabo fish taco,Guacamole,"A house blend of avocado, lime juice, cilantro...",seafood mexican,a house blend avocado lime juice cilantro sign...,guacamole,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,37789,cabo fish taco,Crispy Calamari,"Beer battered calamari rings, flash-fried and ...",seafood mexican,beer batter calamari ring flash fry serve roas...,crispy calamari,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,37789,cabo fish taco,Baja Shrimp and Crab Dip,Creamy dip made with lump crabmeat and seasone...,seafood mexican,creamy dip lump crabmeat seasoned shrimp serve...,baja shrimp crab dip,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,37789,cabo fish taco,Salsa Trio,"Roasted Red Tomato Salsa, fresh Pineapple Mang...",seafood mexican,roasted red tomato salsa fresh pineapple mango...,salsa trio,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
filtered_allmenus_df.reset_index(drop=True, inplace=True)

In [43]:
# Set all newly created columns to 0 by default
filtered_allmenus_df[categories] = filtered_allmenus_df[categories].fillna(0)
filtered_allmenus_df.head(1)

,id,name,menu_titles,menu_desc,categories,clean_menu_desc,clean_menu_titles,chinese,bar,latin,...,dining,irish,nuevo,calzones,wraps,wings,greek,tex-mex,pakistani,american
0,37789,cabo fish taco,Queso Dip,Piping hot melted cheese blended with chipotle...,seafood mexican,pipe hot melted cheese blend chipotle pepper s...,queso dip,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
def set_value_categories(df):
    # Loop through categories column
    for row in df.itertuples():

        filtered_categories = [] # Used for appending categories

        # Create list
        cats = row.categories.split(' ')

        for category in cats:

            # Get all food and drink related categories
            if category in categories:
                filtered_categories.append(category)

        df.loc[row.Index, filtered_categories] = 1 # Set to 1 for all food/drink related categories

In [45]:
set_value_categories(filtered_allmenus_df)

#### Additional text cleaning such as removing extra whitespace, '-pron-', and general columns such as 'food'

In [46]:
# Remove '-pron-' from text values and whitespace
filtered_allmenus_df['clean_menu_desc'] = filtered_allmenus_df['clean_menu_desc']\
    .apply(lambda x: x.replace('-pron-', '')).str.strip()

filtered_allmenus_df['clean_menu_titles'] = filtered_allmenus_df['clean_menu_titles']\
    .apply(lambda x: x.replace('-pron-', '')).str.strip()

# Remove extra white space
filtered_allmenus_df['clean_menu_desc'] = filtered_allmenus_df['clean_menu_desc']\
    .apply(lambda x: re.sub(' +', ' ', x))

filtered_allmenus_df['clean_menu_titles'] = filtered_allmenus_df['clean_menu_titles']\
    .apply(lambda x: re.sub(' +', ' ', x))

In [47]:
# Remove general term 'food' from the list
filtered_allmenus_df = filtered_allmenus_df.drop(columns=['id', 'menu_titles', 'menu_desc', 'categories', 'food'])

# Rename (traditional) and (new) to traiditonal and new
filtered_allmenus_df = filtered_allmenus_df.rename(columns={"(traditional)": "traditional", "(new)": "new"})

In [48]:
filtered_allmenus_df.shape

(313953, 113)

In [49]:
filtered_allmenus_df.reset_index(drop=True, inplace=True)
filtered_allmenus_df.head()

,name,clean_menu_desc,clean_menu_titles,chinese,bar,latin,filipino,crepes,sushi,gyros,...,dining,irish,nuevo,calzones,wraps,wings,greek,tex-mex,pakistani,american
0,cabo fish taco,pipe hot melted cheese blend chipotle pepper s...,queso dip,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,cabo fish taco,a house blend avocado lime juice cilantro sign...,guacamole,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,cabo fish taco,beer batter calamari ring flash fry serve roas...,crispy calamari,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cabo fish taco,creamy dip lump crabmeat seasoned shrimp serve...,baja shrimp crab dip,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,cabo fish taco,roasted red tomato salsa fresh pineapple mango...,salsa trio,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
filtered_allmenus_df[filtered_allmenus_df.isna().any(axis=1)]

,name,clean_menu_desc,clean_menu_titles,chinese,bar,latin,filipino,crepes,sushi,gyros,...,dining,irish,nuevo,calzones,wraps,wings,greek,tex-mex,pakistani,american


#### Create menu description and title dataframes

In [71]:
menu_desc_df = filtered_allmenus_df[['name', 'clean_menu_desc']]
menu_title_df = filtered_allmenus_df[['name', 'clean_menu_titles']]

In [79]:
menu_desc_df.to_csv('./dataset/menu_desc.csv')
menu_title_df.to_csv('./dataset/menu_title.csv')

In [80]:
menu_title_df[menu_title_df.isna().any(axis=1)]

,name,clean_menu_titles


In [ ]:
menu_df.loc[menu_df['clean_menu_titles'].isna(), 'clean_menu_titles'] = menu_df['name']

In [83]:
menu_df[menu_df.isna().any(axis=1)]``

,name,clean_menu_titles


In [84]:
menu_df.to_csv('./dataset/menu_title.csv')

#### Save filtered menu dataset

In [54]:
filtered_allmenus_df.to_csv('./dataset/filtered_menu.csv')

In [56]:
menu_df = pd.read_csv('./dataset/filtered_menu.csv', index_col=0)
menu_df[menu_df.isna().any(axis=1)]

,name,clean_menu_desc,clean_menu_titles,chinese,bar,latin,filipino,crepes,sushi,gyros,...,dining,irish,nuevo,calzones,wraps,wings,greek,tex-mex,pakistani,american
253,phat burrito,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,phat burrito,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7625,dj hollywood's pizza & gyros,NaN,french fry,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
7626,dj hollywood's pizza & gyros,NaN,onion ring,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
7628,dj hollywood's pizza & gyros,NaN,chili cheese fries,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310152,the rack,NaN,mixed veggie,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
310153,the rack,NaN,sauteed spinach,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
310154,the rack,NaN,sweet potato fry,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
310155,the rack,NaN,grilled asparagus,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# Saving food categories for later use
with open('./dataset/food_categories.pkl', 'wb') as f:
    pickle.dump(categories, f)

### Summary of Allmenus dataset
 - Before extracting data through webscraping, I retrieved US states and its respective cities that appears on Yelp's dataset.
 - Used BeautifulSoup in extracting restaurant's menu data from allmenus using Yelp's states and cities information.
 - Filtered restaurants that appeared on both Yelp's business and Allmenus' menu dataset which resulted in collecting 1199 unique restaurants.
     - Restaurants that appeared on both allmenus and Yelp's business dataset resulted in the following:
        - Removed 150,439 restaurant data from Yelp's business dataset (97% reduction).
        - Removed 1,040,494 restaurants from allmenus data (76.8% reduction).
        - Resulted in having 1199 unique restaurants.
 - Menu dataframe contains 313,953 rows with 115 columns consisting of restaurant name, attributes, clean menu description and titles.
 - Created categories columns based on Allmenus' categories values (ex: There are Japanese, Indian columns, and etc.)

---

## 3) Clean Yelp's Business, Review, Tips Datasets
Yelp’s review and tip has rich information about restaurants’ attributes and users’ feelings towards the restaurants they dined in.  Unlike Allmenus’ dataset - it does not have direct food information but rather general text value about the restaurant. Therefore, I thought it’ll be useful in identifying the restaurant using text value provided in reviews and tips. Most of the columns will be removed from the business dataset as new columns will be created using Allmenus' categories columns as it has valuable information regarding restaurant's cuisine. Reviews will be filtered based on restaurants available in the filtered business dataset.

1. Remove all columns except business_id and restaurant name.
2. Filter review  and dataset by business id.
3. Text preprocessing - expanding contractions, tokenizing, lemmatizing, lowercasing, and removing stop words.
4. Define positive and negative review type per review.
    - Define average popularity score based on # of positives / total reviews per restaurant.
4. Create two dataframes
   - Group all reviews/tips by restaurant name and remove duplicates.
   - No grouping - allow duplicate restaurant names as each review is considered unique.
5. Identify cosine similarities between all restaurants.

### 3.1 Remove all columns except business_id and restaurant name

In [130]:
# Remove all columns except business_id and restaurant name
business_df = business_df[['business_id', 'name']].reset_index(drop=True)
business_df.head()

,business_id,name
0,Sd75ucXKoZUM2BEfBHFUOg,china gourmet
1,j9bWpCRwpDVfwVT_V85qeA,papaya thai
2,MTx-Zdl_KcU_z9G832XAjg,taco bell
3,UZ34XAw1GJ3chCOuKL69Mg,garcia's
4,UITPqkoDytnHT4kxaAyDeA,hibachi express


### 3.2 Filter review and dataset by business id
- Remove columns that are not needed for NLP project.
- Remove rows with NaNs.
- Remove all reviews and tips that are not associated with restaurants in filtered business dataframe.
- Create two reviews dataframes:
    - 1) Dataframe with both positive and negative reviews
    - 2) Dataframe with positive reviews only

### Reviews and Tips

In [131]:
review_df.shape

(8021122, 9)

In [132]:
tip_df.shape

(1320761, 5)

In [133]:
review_df.head()

,funny,review_id,useful,date,business_id,text,cool,stars,user_id
0,0,xQY8N_XvtGbearJ5X4QryQ,5,2015-04-15 05:21:16,-MhfebM0QIsKt87iDN-FNw,"As someone who has worked with many museums, I...",0,2.0,OwjRMXRC0KyPrIlcjaXeFQ
1,1,UmFMZ8PyXZTY2QcwzsfQYA,1,2013-12-07 03:16:52,lbrU8StCq3yDfr-QMnGrmQ,I am actually horrified this place is still in...,0,1.0,nIJD_7ZXHq-FX8byPMOkMQ
2,0,LG2ZaYiOgpr2DK_90pYjNw,1,2015-12-05 03:18:11,HQl28KMwrEKHqhFrrDqVNQ,I love Deagan's. I do. I really do. The atmosp...,0,5.0,V34qejxNsCbcgD8C0HVk-Q
3,0,i6g_oA9Yf9Y31qt0wibXpw,0,2011-05-27 05:30:52,5JxlZaqCnk1MnbgRirs40Q,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",0,1.0,ofKDkJKXSKZXu5xJNGiiBQ
4,0,6TdNDKywdbjoTkizeMce8A,0,2017-01-14 21:56:57,IS4cv902ykd8wj1TR0N3-A,"Oh happy day, finally have a Canes near my cas...",0,4.0,UgMW8bLE0QMJDCkQ1Ax5Mg


In [134]:
tip_df.head()

,compliment_count,date,business_id,text,user_id
0,0,2013-11-26 18:20:08,UYX5zL_Xj9WEc_Wp-FrqHw,Here for a quick mtg,hf27xTME3EiCp6NL6VtWZQ
1,0,2014-06-15 22:26:45,Ch3HkwQYv1YKw_FO06vBWA,Cucumber strawberry refresher,uEvusDwoSymbJJ0auR3muQ
2,0,2016-07-18 22:03:42,rDoT-MgxGRiYqCmi0bG10g,Very nice good service good food,AY-laIws3S7YXNl_f_D6rQ
3,0,2014-06-06 01:10:34,OHXnDV01gLokiX1ELaQufA,It's a small place. The staff is friendly.,Ue_7yUlkEbX4AhnYdUfL7g
4,0,2011-04-08 18:12:01,GMrwDXRlAZU2zj5nH6l4vQ,"8 sandwiches, $24 total...what a bargain!!! An...",LltbT_fUMqZ-ZJP-vJ84IQ


#### 3.2.1 Remove columns that are not needed for NLP project

In [138]:
# Remove all except business_id and text
review_df = review_df[['business_id', 'text']]

In [139]:
tip_df = tip_df[['business_id', 'text']]

#### 3.2.2 Remove rows with NaNs

In [140]:
# Check for missing values
review_df.isna().sum()

business_id    0
text           2
dtype: int64

In [141]:
tip_df.isna().sum()

business_id    0
text           8
dtype: int64

In [142]:
review_df[review_df['text'].isna()]

,business_id,text
3529709,oXoVJ0xKv82cBo9U6oEjlQ,NaN
6502433,I7XWtKL1oHohZCtCCyQIog,NaN


In [143]:
# Dropping rows with NaN as I can't do much without text involved
review_df.dropna(inplace=True)
review_df.reset_index(inplace=True)

tip_df.dropna(inplace=True)
tip_df.reset_index(inplace=True)

#### 3.2.3 Remove all reviews and tips that are not associated with restaurants in filtered business dataframe.

In [144]:
# Get reviews that are in business dataframe
m = review_df['business_id'].isin(business_df['business_id'])
review_df = review_df[m].reset_index(drop=True)
review_df.shape

(367770, 3)

In [145]:
# Get tips that are in business dataframe
m = tip_df['business_id'].isin(business_df['business_id'])
tip_df = tip_df[m].reset_index(drop=True)
tip_df.shape

(70834, 3)

Dropped reviews from 8,021,122 to 367,770 rows (95.5% reduction) and dropped tips from 1,320,761 to 70,834 (95% reduction)

#### 3.2.4 Create two review dataframes (one with all reviews and other without negative reviews)
Positive reviews may tend have better description of the restaurant and its attributes compared to negative reviews. For instance, negative reviews may only provide 'wait was too long' while positive review may contain 'steak was delicious' in which 'steak' can be used as keyword for nlp. Since I can't prove whether having only postive reviews will provide better search result, I am creating both so that it can be tested later on during machine learning step.

I am defining text value as positive or negative based on given compound score using **Vadar's sentiment analysis**.

The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).

- positive sentiment : (compound score >= 0.05)
- neutral sentiment : (compound score > -0.05) and (compound score < 0.05)
- negative sentiment : (compound score <= -0.05)

In [146]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [147]:
# Get sentiment score
review_df['sentiment_result'] = pd.DataFrame(review_df.apply(lambda x: analyser.polarity_scores(x['text']), axis = 1))

# Normalize the sentiment_result 
norm_sentiment_df = pd.json_normalize(review_df['sentiment_result'])
review_df = review_df.join(norm_sentiment_df)

# Drop sentiment_result column
review_df.drop(columns = ['sentiment_result'],inplace = True)
review_df.head()

,index,business_id,text,neg,neu,pos,compound
0,16,d4qwVw4PcN-_2mK2o1Ro1g,10pm on a super bowl Sunday and they're alread...,0.277,0.565,0.158,-0.2406
1,21,d4qwVw4PcN-_2mK2o1Ro1g,Holy heck this place is amazing. I love their ...,0.000,0.484,0.516,0.9648
2,27,wkzWdo1mBqbzR2KPoXtWZw,Was a Chicago style deep dish. Homemade type ...,0.000,0.669,0.331,0.8860
3,55,VMPSdoBgJuyS9t_x_caTig,Easily my favorite place to eat in Madison. G...,0.031,0.697,0.272,0.9451
4,65,vhKUBKcOm8zOLxeS3XM9iA,More than just Pizza. This location is small ...,0.026,0.868,0.107,0.9186


In [149]:
# Get sentiment score
tip_df['sentiment_result'] = pd.DataFrame(tip_df.apply(lambda x: analyser.polarity_scores(x['text']), axis = 1))

# Normalize the sentiment_result 
norm_sentiment_df = pd.json_normalize(tip_df['sentiment_result'])
tip_df = tip_df.join(norm_sentiment_df)

# Drop sentiment_result column
tip_df.drop(columns = ['sentiment_result'],inplace = True)
tip_df.head()

,index,business_id,text,neg,neu,pos,compound
0,1,Ch3HkwQYv1YKw_FO06vBWA,Cucumber strawberry refresher,0.0,1.000,0.000,0.0000
1,3,OHXnDV01gLokiX1ELaQufA,It's a small place. The staff is friendly.,0.0,0.652,0.348,0.4939
2,4,GMrwDXRlAZU2zj5nH6l4vQ,"8 sandwiches, $24 total...what a bargain!!! An...",0.0,0.498,0.502,0.7946
3,6,d_L-rfS1vT3JMzgCUGtiow,Cochinita Pibil was memorable & delicious !,0.0,0.501,0.499,0.6114
4,8,rcaPajgKOJC2vo_l3xa42A,Kelly is an awesome waitress there!,0.0,0.532,0.468,0.6588


In [150]:
review_df = review_df.drop(columns=['index'])
tip_df = tip_df.drop(columns=['index'])

### 3.3 Text Preprocessing
- Expanding contractions, Tokenizing, Lemmatizing, lowerasing, and removing stop words.

In [151]:
# Expanding contractions - ex: 'y'all can't' => 'you all cannot'
review_df['clean_text'] = review_df['text'].apply(lambda text: expand_contractions(text))
tip_df['clean_text'] = tip_df['text'].apply(lambda text: expand_contractions(text))

In [152]:
review_df['clean_text'] = review_df['text'].apply(lambda text: tokenize(text))

In [153]:
tip_df['clean_text'] = tip_df['text'].apply(lambda text: tokenize(text))

### 3.4 Create four dataframes
- Group all reviews/tips by restaurant name and remove duplicates.
    - Concat both tips and reviews dataframe and groupby business_id and combine all texts.
    - Add restaurant name to combined reviews and tips dataframe by using business dataframe's id.
        - Groupby restaurant name and combine text again.
    - Positive reviews' text values grouped together based on Yelp's business_id.
    - All reviews' text values grouped together based on Yelp's business_id.
- No grouping - allow duplicate restaurant names as each review is considered unique.
    - Positive reviews only
    - All reviews

#### 3.4.1 Group all reviews/tips by resaurant name and remove duplicates
Creating two dataframes - one consisting unique restaurant with consolidated text values.
- Concat both tips and reviews dataframe and groupby business_id and combine all texts
- Define positive and negative review based on compound score.
- Define restaurant popularity based on average sentiment compound score.
- Set restaurant name per review based on Yelp's business_id.

In [154]:
# Concat reviews and tips
text_df = pd.concat([review_df, tip_df])

In [155]:
text_df.shape

(438604, 7)

In [156]:
text_df = text_df[['business_id', 'text', 'clean_text', 'compound']]
text_df.head(3)

,business_id,text,clean_text,compound
0,d4qwVw4PcN-_2mK2o1Ro1g,10pm on a super bowl Sunday and they're alread...,p.m. super bowl sunday close weak wonder hard ...,-0.2406
1,d4qwVw4PcN-_2mK2o1Ro1g,Holy heck this place is amazing. I love their ...,holy heck -pron- chicken taco far favorite gre...,0.9648
2,wkzWdo1mBqbzR2KPoXtWZw,Was a Chicago style deep dish. Homemade type ...,be chicago style deep dish homemade type crust...,0.8860


#### Define positive and negative review based on compound score
- positive sentiment : (compound score >= 0.05)
- neutral sentiment : (compound score > -0.05) and (compound score < 0.05)
- negative sentiment : (compound score <= -0.05)

In [155]:
text_df['positive'] = 0 # Setting as 0 by default as negative review

# Setting as positive review if compound score is greater than -0.05
text_df.loc[test_df['compound'] > -0.05, 'positive'] = 1

#### Define popularity score by averaging # of positive reviews divided by review occurrences (mean)

In [156]:
text_df['popularity_score'] = text_df.groupby('business_id')['positive']\
    .transform(lambda x: round(x.mean(),2) * 100)

In [157]:
text_df.head(5)

,business_id,text,clean_text,compound,name,positive,popularity_score
0,d4qwVw4PcN-_2mK2o1Ro1g,10pm on a super bowl Sunday and they're alread...,p.m. super bowl sunday close weak wonder hard ...,-0.2406,pink taco,0,87.0
1,d4qwVw4PcN-_2mK2o1Ro1g,Holy heck this place is amazing. I love their ...,holy heck -pron- chicken taco far favorite gre...,0.9648,pink taco,1,87.0
2,wkzWdo1mBqbzR2KPoXtWZw,Was a Chicago style deep dish. Homemade type ...,be chicago style deep dish homemade type crust...,0.8860,vito's pizza,1,91.0
3,VMPSdoBgJuyS9t_x_caTig,Easily my favorite place to eat in Madison. G...,easily favorite madison great lao thai curry f...,0.9451,lao laan-xang restaurant,1,95.0
4,vhKUBKcOm8zOLxeS3XM9iA,More than just Pizza. This location is small ...,more pizza this location small cozy have large...,0.9186,rosati's pizza,1,71.0


In [158]:
# Drop compound score  and text column
text_df = text_df.drop(columns=['compound', 'text'], axis=1)

#### Set restaurant name per review based on Yelp's business_id 

In [159]:
text_df['name'] = text_df['business_id'].apply(lambda x: business_df[business_df['business_id'] == x]['name']\
                                               .values[0])

In [160]:
text_df.head(3)

,business_id,clean_text,name,positive,popularity_score
0,d4qwVw4PcN-_2mK2o1Ro1g,p.m. super bowl sunday close weak wonder hard ...,pink taco,0,87.0
1,d4qwVw4PcN-_2mK2o1Ro1g,holy heck -pron- chicken taco far favorite gre...,pink taco,1,87.0
2,wkzWdo1mBqbzR2KPoXtWZw,be chicago style deep dish homemade type crust...,vito's pizza,1,91.0


In [161]:
# Re-order columns
text_df = text_df[['business_id', 'name', 'clean_text', 'positive', 'popularity_score']]

In [184]:
# Remove '-pron-' from text values and whitespace
text_df['clean_text'] = text_df['clean_text'].apply(lambda x: x.replace('-pron-', '')).str.strip()

# Remove extra white space
text_df['clean_text'] = text_df['clean_text'].apply(lambda x: re.sub(' +', ' ', x))

In [185]:
# Convert str type
text_df['clean_text'] = text_df['clean_text'].astype(str)

#### Create 3 additional dataframes
- Positive reviews only
- Positive reviews grouped by Yelp's business_id
- All reviews grouped by Yelp's business_id

In [209]:
# Get positive reviews only
pos_df = text_df[text_df['positive'] == 1]

# Group all the positive reviews' text values together based on yelp's business_id
grouped_pos_df = pos_df.groupby('name', as_index=False).agg(
    {'clean_text': ' '.join, 
     'popularity_score': lambda x: round(x.mean(),2)
    })

# Group all reviews' text values together based on yelp's business_id
grouped_text_df = text_df.groupby('name', as_index=False).agg(
    {'clean_text': ' '.join, 
     'popularity_score': lambda x: round(x.mean(),2)
    })

In [210]:
# Remove 'positive' column
pos_df = pos_df.drop(columns=['positive', 'business_id'], axis=1)
pos_df.head()

,name,clean_text,popularity_score
1,pink taco,holy heck chicken taco far favorite great cust...,87.0
2,vito's pizza,be chicago style deep dish homemade type crust...,91.0
3,lao laan-xang restaurant,easily favorite madison great lao thai curry f...,95.0
4,rosati's pizza,more pizza this location small cozy have large...,71.0
5,bouchon,bouchon favorite hand extremly fresh eat bouch...,93.0


In [211]:
grouped_pos_df

,name,clean_text,popularity_score
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,78.00
1,24th street pizza & gyros,gyro pizza pretty fantastic thing keep come de...,80.00
2,3 tomatoes & a mozzarella,absolutely delicious customer go today single ...,76.19
3,4b cafe,good consensus average home fairly easily emul...,100.00
4,5 r cha thai go,yum yum yum try dynamite fried rice amazing fi...,83.00
...,...,...,...
1193,z tejas,good dinner horrible el hefe decide margaritas...,90.00
1194,z'tejas,great spot dinner love diablo pasta fajita sel...,89.00
1195,zaiaka,this favorite indian try pittsburgh area the f...,93.00
1196,zhen choo chinese restaurant,live area order zhen choo year chinese hit mis...,92.00


In [213]:
grouped_text_df

,name,clean_text,popularity_score
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,78.00
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,80.00
2,3 tomatoes & a mozzarella,absolutely delicious customer go today single ...,72.52
3,4b cafe,good consensus average home fairly easily emul...,100.00
4,5 r cha thai go,try r cha today turn permanently close very sa...,83.00
...,...,...,...
1194,z tejas,good dinner horrible el hefe decide margaritas...,90.00
1195,z'tejas,great spot dinner love diablo pasta fajita sel...,89.00
1196,zaiaka,this favorite indian try pittsburgh area the f...,93.00
1197,zhen choo chinese restaurant,live area order zhen choo year chinese hit mis...,92.00


In [218]:
print(f'All reviews dataframe: {text_df.shape[0]} rows')
print(f'Positive reviews dataframe: {pos_df.shape[0]} rows')
print(f'Grouped positive reviews dataframe: {grouped_pos_df.shape[0]} rows')
print(f'Grouped reviews dataframe: {grouped_text_df.shape[0]} rows')

All reviews dataframe: 438604 rows
Positive reviews dataframe: 373687 rows
Grouped positive reviews dataframe: 1198 rows
Positive reviews dataframe: 1199 rows


In [224]:
text_count = text_df['clean_text'].str.len().sum()
pos_text_count = pos_df['clean_text'].str.len().sum()
grouped_pos_count = grouped_pos_df['clean_text'].str.len().sum()
grouped_text_count = grouped_text_df['clean_text'].str.len().sum()

print(f'All reviews dataframe text count: {text_count}')
print(f'Positive reviews dataframe text count: {pos_text_count}')
print(f'Grouped positive reviews dataframe text count: {grouped_pos_count}')
print(f'Grouped reviews dataframe text count: {grouped_text_count}')

All reviews dataframe text count: 101201300
Positive reviews dataframe text count: 86448479
Grouped positive reviews dataframe text count: 86820968
Grouped reviews dataframe text count: 101638705


In [88]:
text_df = text_df[['name', 'clean_text', 'popularity_score']]

#### Saving files

In [89]:
text_df.to_csv('./dataset/text.csv') # All reviews
pos_df.to_csv('./dataset/positive_text.csv') # Positive reviews
grouped_pos_df.to_csv('./dataset/grouped_positive_text.csv') # Grouped positive text values per restaurant
grouped_text_df.to_csv('./dataset/grouped_text.csv') # Grouped text values per restaurant

### 3.5 Identify cosine similarities between all restaurants
- Aggregate all text values from 'menu' dataframe grouped by restaurant name.
- Merge menu dataframe and grouped_text dataframe on 'name' column.
- Add menu attributes' columns to merged dataframe based on 'name'.

#### 3.5.1 Aggregate all text values from 'menu' dataframe grouped by restaurant name

In [230]:
# Consolidate all menu description and menu titls using groupby method
menu_df = filtered_allmenus_df.groupby(['name'], as_index=False)\
    .agg({'clean_menu_desc': ' '.join, 
          'clean_menu_titles': ' '.join
         })

menu_df.head()

,name,clean_menu_desc,clean_menu_titles
0,#1 pho,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...
1,24th street pizza & gyros,tomato onion tzatziki lettuce tomato onion tza...,gyros pita chicken gyros pita calzone fried zu...
2,3 tomatoes & a mozzarella,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...
3,4b cafe,chip and salsa chip and nacho cheese sauce chi...,chip and salsa chip and nacho cheese sauce chi...
4,5 r cha thai go,thai iced tea thai iced coffee iced tea soft d...,thai iced tea thai iced coffee iced tea soft d...


In [232]:
menu_df.shape

(1199, 3)

#### 3.5.2 Merge menu dataframe and grouped_text_dataframe on 'name' column

In [234]:
merged_df = grouped_text_df.merge(menu_df, on='name')
merged_df.head(3)

,name,clean_text,popularity_score,clean_menu_desc,clean_menu_titles
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,78.00,finely ground pork grill wooden skewer lettuce...,grilled meat ball vietname crispy crepe crispy...
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,80.00,tomato onion tzatziki lettuce tomato onion tza...,gyros pita chicken gyros pita calzone fried zu...
2,3 tomatoes & a mozzarella,absolutely delicious customer go today single ...,72.52,small salad dress house balsamic vinaigrette r...,field greens caesar mediterranean spinach arug...


In [235]:
# remove additional space from string  
merged_df['clean_text'] = merged_df['clean_text'].apply(lambda text: re.sub(' +', ' ', text))
merged_df['clean_menu_desc'] = merged_df['clean_menu_desc'].apply(lambda text: re.sub(' +', ' ', text))
merged_df['clean_menu_titles'] = merged_df['clean_menu_titles'].apply(lambda text: re.sub(' +', ' ', text))

In [242]:
# Re-order columns
merged_df = merged_df[['name','clean_text', 'clean_menu_titles', 'clean_menu_desc','popularity_score']]

#### 3.5.3 Add menu attributes' columns to merged dataframe based on 'name'.

In [240]:
df = filtered_allmenus_df.drop_duplicates(subset=['name'], keep='first')
df.reset_index(drop=True, inplace=True)
df.drop(columns=['clean_menu_desc', 'clean_menu_titles'], inplace=True)
df.head()

,name,gyros,european,cantonese,cajun,jamaican,specials,fruit,menu,french,...,cream,tex-mex,halal,vegetarian,latin,vietnamese,british,ice,pub,bar
0,cabo fish taco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,monterrey,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,phat burrito,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,tres pesos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,las margaritas,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [244]:
merged_df = merged_df.merge(df, on='name')
merged_df.head(3)

,name,clean_text,clean_menu_titles,clean_menu_desc,popularity_score,gyros,european,cantonese,cajun,jamaican,...,cream,tex-mex,halal,vegetarian,latin,vietnamese,british,ice,pub,bar
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,grilled meat ball vietname crispy crepe crispy...,finely ground pork grill wooden skewer lettuce...,78.00,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,gyros pita chicken gyros pita calzone fried zu...,tomato onion tzatziki lettuce tomato onion tza...,80.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,absolutely delicious customer go today single ...,field greens caesar mediterranean spinach arug...,small salad dress house balsamic vinaigrette r...,72.52,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Combine clean menu description and titles together

In [246]:
merged_df['menu'] = merged_df['clean_menu_desc'] + merged_df['clean_menu_titles']

# remove additional space from string  
merged_df['menu'] = merged_df['menu'].apply(lambda text: re.sub(' +', ' ', text))

# Drop unneeded columns
merged_df = merged_df.drop(columns=['clean_menu_desc', 'clean_menu_titles'])

# Re-order columns
first_cols = ['name', 'clean_text', 'menu']
second_cols = []
for col in merged_df.columns:
    if col not in first_cols:
        second_cols.append(col)
    
merged_df = merged_df[first_cols + second_cols]

# final dataset
merged_df.head()

,name,clean_text,menu,popularity_score,gyros,european,cantonese,cajun,jamaican,specials,...,cream,tex-mex,halal,vegetarian,latin,vietnamese,british,ice,pub,bar
0,#1 pho,fantastic pho vegetable pho vegetable broth fr...,finely ground pork grill wooden skewer lettuce...,78.00,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,24th street pizza & gyros,change ownership stop day advertise pizza slic...,tomato onion tzatziki lettuce tomato onion tza...,80.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3 tomatoes & a mozzarella,absolutely delicious customer go today single ...,small salad dress house balsamic vinaigrette r...,72.52,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4b cafe,good consensus average home fairly easily emul...,chip and salsa chip and nacho cheese sauce chi...,100.00,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,5 r cha thai go,try r cha today turn permanently close very sa...,thai iced tea thai iced coffee iced tea soft d...,83.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
merged_df[merged_df.isna().any(axis=1)]

,name,clean_text,menu,popularity_score,gyros,european,cantonese,cajun,jamaican,specials,...,cream,tex-mex,halal,vegetarian,latin,vietnamese,british,ice,pub,bar


In [248]:
merged_df.to_csv('./dataset/restaurants_data.csv')

In [204]:
# Create categories dataframe
merged_df = merged_df[['name']]
categories_df = merged_df.merge(df, on='name')
categories_df[categories_df['name'] == "vito's pizza"]['chinese']

1157    0
Name: chinese, dtype: int64

In [ ]:
categories_df = categories_df.drop(columns=['id', 'menu_titles', 'menu_desc', 'categories'])

In [210]:
categories_df.to_csv('./dataset/categories.csv')

### More text preprocessing
Found several words containing stop words and other single values that does not contribute to overall search engine. Below are following steps to further clean the text values
- Remove stop words that were populated from previous text preprocessing.
- Remove rows that contain non-english characters.
- Identify rows with less than 5 tokens and remove if they don't contain restaurant keywords such as 'chinese, sushi, steak, etc.'.

In [ ]:
# Organize cuisine dataframes by utilizing dictionary.
categories_dict = {}
cols = categories_df.columns.tolist()
dfs = [] # Create master cuisine list

# Gather cuisine and key it by cuisine type onto dictionary
for col in cols:
    category_df = categories_df[categories_df[col] == 1]
    categories_dict[col] = category_df.reset_index(drop=True)

In [51]:
# Defines whether the text is part of categorical value or not
def is_categorical(text):
    
    for word in text.split():
        if word in categories_dict:
            return True
    
    return False

# Remove non-english words
def is_en(text):
    is_english = re.search("[a-zA-Z\s]+", text)
    
    if is_english is None:
        return False
    else:
        return True

In [ ]:
text_df.shape

In [ ]:
# Remove rows with empty string
text_df = text_df[text_df['clean_text'] != '']

# Remove rows containing non-english characters
text_df = text_df[text_df['clean_text'].apply(is_en)]
text_df.reset_index(drop=True, inplace=True)

# Identify rows with 5 tokens or less and remove only if they don't any restaurant related text values
df = text_df[text_df['clean_text'].str.split().str.len().lt(5)]

# Filter by category
df = df[df['clean_text'].apply(is_categorical)]
filtered_indices = list(df.index.values)
indices_for_removal = [i for i in df.index.values if i not in filtered_indices]

text_df = text_df.drop(indices_for_removal)
text_df.reset_index(drop=True, inplace=True)
text_df.shape

In [ ]:
text_df.to_csv('./dataset/clean_text.csv')

### Summary of Yelp Dataset
Most of the columns in Yelp's business, review, and tips dataframes have been removed except for business_id and text column. Grouped both review and tips dataframe and began grouping based on business_id and restaurant name which resulted in consolidating all text into one row per restaurant. After removing duplicates on all dataframes, I merged consolidated yelp's dataframe with allmenus dataframe. Merged data frame has all the necessary information in identifying the restaurant as it has menu description, menu titles, restaurant cuisine attributes, and users inputted text.

**Snapshot**
1. Dropped reviews from 8,021,122 to 367,770 rows (95.5% reduction) and dropped tips from 1,320,761 to 70,834 (95% reduction)
2. Combined reviews and tips dataframe which accumulated up to 438,604; however after grouping based on business_id and restaurant name - it dropped to 1199.
3. Dropped rows with NaNs in the text column.
4. Merged cleaned Yelp's dataframe with Allmenus dataframe on restaurant name to be used for EDA.
5. Tokenized all text and removed any whitespace occurring on text columns.
6. Created 4 text dataframes:
    - All reviews dataframe: 438604 rows with 101201300 word count.
    - Positive reviews dataframe: 373687 rows with 86448479 word count.
    - Grouped positive reviews dataframe: 1198 rows with 86820968 word count.
    - Grouped reviews dataframe: 1199 rows with 101638705 word count.

    Created all four dataframes to see how well it performs with doc2vec algorithm. 